In [3]:
import os

In [4]:
import json
import pandas as pd
import traceback

In [5]:
import langchain

In [6]:
from langchain.chat_models import ChatCohere

In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
KEY = os.getenv("Cohere_api_key")

In [9]:
print(KEY)

p5Th2oebdkjfebVUDBJInWOEyPPIjThAvj8EpaNS


In [10]:
llm = ChatCohere(cohere_api_key = KEY, temperature = 0.5)

c:\Users\sarga\MCQGenerator\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.cohere.ChatCohere` was deprecated in langchain-community 0.0.30 and will be removed in 0.2.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import ChatCohere`.
  warn_deprecated(


In [11]:
llm

ChatCohere(client=<cohere.client.Client object at 0x0000020916639EB0>, async_client=<cohere.client.AsyncClient object at 0x0000020916FA70D0>, temperature=0.5, cohere_api_key=SecretStr('**********'))

In [12]:
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import StdOutCallbackHandler
import PyPDF2

In [13]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    }
}

In [14]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide.
Ensure to make {number} MCQs
{response_json}

"""

In [15]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [16]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [17]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [18]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [19]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [33]:
generate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],output_variables=["quiz", "review"], verbose=True,)

In [34]:
file_path = r"C:\Users\sarga\MCQGenerator\data.txt"

In [35]:
print(file_path)

C:\Users\sarga\MCQGenerator\data.txt


In [36]:

with open(file_path, 'r') as file:
    TEXT = file.read()

 

In [37]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contributed

In [38]:
#Serialize the python dictionary into a JSON-fromatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [39]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [41]:
# Create an instance of StdOutCallbackHandler
stdout_callback = StdOutCallbackHandler()

# Generate evaluation chain using the callback instance
response=generate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used 

AttributeError: 'NonStreamedChatResponse' object has no attribute 'token_count'

In [30]:
response

NameError: name 'response' is not defined

In [ ]:
quiz = response.get("quiz")

In [ ]:
quiz1 = quiz.replace('```', '')
print(quiz1)

json
{
  "1": {
    "mcq": "Who coined the term 'Machine Learning' and in which year?",
    "options": {
      "a": "Arthur Samuel, 1950",
      "b": "Alan Turing, 1959",
      "c": "Donald Hebb, 1949",
      "d": "Arthur Samuel, 1959"
    },
    "correct": "d"
  },
  "2": {
    "mcq": "What was the first machine learning model invented by Arthur Samuel about?",
    "options": {
      "a": "Playing checkers",
      "b": "Classifying patterns",
      "c": "Predicting stock market trends",
      "d": "Analyzing human brain functions"
    },
    "correct": "a"
  },
  "3": {
    "mcq": "Which company developed an early 'learning machine' called Cybertron?",
    "options": {
      "a": "IBM",
      "b": "Raytheon",
      "c": "Cohere",
      "d": "Microsoft"
    },
    "correct": "b"
  },
  "4": {
    "mcq": "What is the primary objective of modern-day machine learning?",
    "options": {
      "a": "To develop algorithms that can beat humans in games",
      "b": "Classifying data and maki

In [ ]:
start_index = quiz1.find('{')

# Remove everything before the first '{' character
cleaned_json = quiz1[start_index:]

# Load the cleaned JSON data into a Python dictionary
data = json.loads(cleaned_json)

# Print the cleaned JSON data
quiz2 = json.dumps(data, indent=2)
print(quiz2)

{
  "1": {
    "mcq": "Who coined the term 'Machine Learning' and in which year?",
    "options": {
      "a": "Arthur Samuel, 1950",
      "b": "Alan Turing, 1959",
      "c": "Donald Hebb, 1949",
      "d": "Arthur Samuel, 1959"
    },
    "correct": "d"
  },
  "2": {
    "mcq": "What was the first machine learning model invented by Arthur Samuel about?",
    "options": {
      "a": "Playing checkers",
      "b": "Classifying patterns",
      "c": "Predicting stock market trends",
      "d": "Analyzing human brain functions"
    },
    "correct": "a"
  },
  "3": {
    "mcq": "Which company developed an early 'learning machine' called Cybertron?",
    "options": {
      "a": "IBM",
      "b": "Raytheon",
      "c": "Cohere",
      "d": "Microsoft"
    },
    "correct": "b"
  },
  "4": {
    "mcq": "What is the primary objective of modern-day machine learning?",
    "options": {
      "a": "To develop algorithms that can beat humans in games",
      "b": "Classifying data and making pr

In [ ]:
quiz_data = json.loads(quiz2)

In [ ]:
quiz_table_data = []
for key, value in quiz_data.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

[{'MCQ': "Who coined the term 'Machine Learning' and in which year?",
  'Choices': 'a: Arthur Samuel, 1950 | b: Alan Turing, 1959 | c: Donald Hebb, 1949 | d: Arthur Samuel, 1959',
  'Correct': 'd'},
 {'MCQ': 'What was the first machine learning model invented by Arthur Samuel about?',
  'Choices': 'a: Playing checkers | b: Classifying patterns | c: Predicting stock market trends | d: Analyzing human brain functions',
  'Correct': 'a'},
 {'MCQ': "Which company developed an early 'learning machine' called Cybertron?",
  'Choices': 'a: IBM | b: Raytheon | c: Cohere | d: Microsoft',
  'Correct': 'b'},
 {'MCQ': 'What is the primary objective of modern-day machine learning?',
  'Choices': 'a: To develop algorithms that can beat humans in games | b: Classifying data and making predictions | c: Studying human cognitive processes exclusively | d: Only analyzing patterns in data',
  'Correct': 'b'},
 {'MCQ': 'Which of the following best describes the contribution of Donald Hebb to machine learni

In [ ]:
pd.DataFrame(quiz_table_data)

,MCQ,Choices,Correct
0,Who coined the term 'Machine Learning' and in ...,"a: Arthur Samuel, 1950 | b: Alan Turing, 1959 ...",d
1,What was the first machine learning model inve...,a: Playing checkers | b: Classifying patterns ...,a
2,Which company developed an early 'learning mac...,a: IBM | b: Raytheon | c: Cohere | d: Microsoft,b
3,What is the primary objective of modern-day ma...,a: To develop algorithms that can beat humans ...,b
4,Which of the following best describes the cont...,a: He invented the first neural network | b: H...,b


In [ ]:
quiz_final = pd.DataFrame(quiz_table_data)

In [ ]:
quiz_final.to_csv("C:\MCQGenerator\experiment\machinelearning.csv",index=False)

<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
C:\Users\sarga\AppData\Local\Temp\ipykernel_26180\2057481574.py:1: SyntaxWarning: invalid escape sequence '\M'
  quiz_final.to_csv("C:\MCQGenerator\experiment\machinelearning.csv",index=False)


In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'04_07_2024_13_23_18'